### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import time
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error


### Import and rename Dataset columns

In [3]:
dataframe = pd.read_csv('/media/luigi/966AA7066AA6E26B/Users/Luigi/Documents/UACSA/TCC/git/tcc-luigi/features_matrix.csv')

dataframe.rename(columns={'2':'Heart Rate',
                '3' : 'Pulse Width',
                '4' : 'Reflection Index',
                '5' : 'LASI',
                '6' : 'Inflection Point Area',
                '7' : 'Crest Time',
                '8' : 'mnpv',
                '9' : 'SDNN',
                '10' : 'rMSSD',
                '11' : 'pNN50',
                '12' : 'LF',
                '13' : 'HF',
                '14' : 'SBP',
                '15' : 'DBP',
                '0' : 'MBP'},
                inplace=True)

dataframe = dataframe.drop(["1"], axis=1)

dataframe.head(10)

,Heart Rate,Pulse Width,Reflection Index,LASI,Inflection Point Area,Crest Time,mnpv,SDNN,rMSSD,pNN50,LF,HF,SBP,DBP,MBP
0,106,0.187,0.86467,30.303,0.96468,5.136,0.71604,8.1104,10.9540,0.00000,1103.00,98.378,121,66,87
1,100,0.201,0.88897,34.483,0.99873,5.136,0.69843,7.3290,9.5618,0.00000,1007.90,81.042,122,70,91
2,99,0.184,0.84877,28.571,0.97500,5.128,0.70604,5.3333,9.3808,0.00000,1098.80,97.302,103,61,80
3,100,0.187,0.90476,38.462,1.06420,5.136,0.71340,6.0474,9.0711,0.00000,1046.90,95.078,129,71,93
4,103,0.177,0.97091,166.670,0.72344,4.568,0.71074,114.5300,135.0300,0.12500,919.65,60.464,107,73,89
5,95,0.201,0.87686,30.303,1.04120,5.136,0.69164,7.3290,10.4740,0.00000,990.19,73.262,117,77,96
6,122,0.199,0.98812,166.670,2.41340,5.128,0.70142,141.0300,149.9500,0.44444,1234.90,64.061,111,74,89
7,141,0.179,0.88658,35.714,1.09880,5.136,0.71819,153.9300,171.0800,0.30000,1220.60,77.470,139,106,100
8,138,0.373,0.94725,30.303,0.82825,5.016,0.63903,147.1400,171.8200,0.36364,1339.50,45.860,110,94,102
9,100,0.189,0.89994,33.333,1.13260,4.616,0.71618,8.4327,14.1420,0.00000,1072.10,107.520,133,106,103


### Convert and split data into training and test set

In [4]:
# Use numpy to convert to arrays
print('Total dataframe size:', dataframe.shape)
# Labels are the values we want to predict
labels_df = dataframe[['SBP', 'DBP', 'MBP']]
labels = np.array(labels_df) # Remove the labels from the features
Y = labels
print('Labels dataframe size:', labels_df.shape)

# axis 1 refers to the columns
features_df = dataframe.drop(['SBP', 'DBP', 'MBP'], axis = 1)# Saving feature names for later use
features = np.array(features_df)
X = preprocessing.scale(features)
print('Features dataframe size:', features_df.shape)

Total dataframe size: (8442, 15)
Labels dataframe size: (8442, 3)
Features dataframe size: (8442, 12)


In [5]:
paper_metrics = pd.DataFrame({"SBP(ME)" : [0.17 ,-0.71, -0.17, 0.09],
                              "SBP(MAE)" : [16.12, 13.87, 10.29, 8.22],
                              "SBP(STD)" : [10.03, 15.29, 10.35, 10.38],
                              "DBP(ME)" :[-0.11, 0.17, -0.14, 0.23],
                              "DBP(MAE)" :[7.04, 6.82, 5.77, 4.17],
                              "DBP(STD)" :[5.81, 8.91, 5.43, 4.22],
                              "MBP(ME)" :[0.13, 0.02, 0.07, -0.02],
                              "MBP(MAE)" :[8.89, 8.18, 6.38, 4.58],
                              "MBP(STD)" :[6.25, 9.02, 6.62, 5.53]},
                              index = ['LiR', 'DT', 'RF', 'AB'])

paper_metrics

,SBP(ME),SBP(MAE),SBP(STD),DBP(ME),DBP(MAE),DBP(STD),MBP(ME),MBP(MAE),MBP(STD)
LiR,0.17,16.12,10.03,-0.11,7.04,5.81,0.13,8.89,6.25
DT,-0.71,13.87,15.29,0.17,6.82,8.91,0.02,8.18,9.02
RF,-0.17,10.29,10.35,-0.14,5.77,5.43,0.07,6.38,6.62
AB,0.09,8.22,10.38,0.23,4.17,4.22,-0.02,4.58,5.53


### Cria um dataframe para guardar as métricas 


In [6]:
# Cria um dataframe para guardar as métricas avaliadas

metrics = pd.DataFrame(index = ['LiR', 'DT', 'RF', 'AB'],
                       columns = ['SBP(ME)', 'SBP(MAE)', 'SBP(STD)',
                                  'DBP(ME)', 'DBP(MAE)', 'DBP(STD)',
                                  'MBP(ME)', 'MBP(MAE)', 'MBP(STD)'
                                   ])



metrics

,SBP(ME),SBP(MAE),SBP(STD),DBP(ME),DBP(MAE),DBP(STD),MBP(ME),MBP(MAE),MBP(STD)
LiR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
RF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Função para calcular o erro percentual acumulado

In [28]:
def cumulative_error_percentage(array, set_point):
        return 100*len(array[array < set_point])/len(array)

### Train and evaluate Linear Regressor

In [8]:
from sklearn.model_selection import KFold

# Anotações
# Y[:, 0] => Systolic Blood Pressure
# Y[:, 1] => Diastolic Blood Pressure

import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

# Declaração dos vetores para armazenar as métricas
SBP_ME = []
SBP_MAE = []
SBP_STD = []
SBP_CORR = []
SBP_CEP5 = []
SBP_CEP10 = []
SBP_CEP15 = []
# SBP_RMSE = []

DBP_ME = []
DBP_MAE = []
DBP_STD = []
DBP_CORR = []
DBP_CEP5 = []
DBP_CEP10 = []
DBP_CEP15 = []
# DBP_RMSE = []

MBP_ME = []
MBP_MAE = []
MBP_STD = []
MBP_CORR = []
MBP_CEP5 = []
MBP_CEP10 = []
MBP_CEP15 = []
# MBP_RMSE = []

# Aqui a gente deve especificar o modelo a ser testado
model = linear_model.LinearRegression()

# Definição do conjunto de validação cruzada pelo metodo KFold
cv = KFold(n_splits=10) #  random_state=42, shuffle=False

for train_index, test_index in cv.split(X):
    # Separo os conjuntos de treino e de teste
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], Y[train_index], Y[test_index]
    # Treino o modelo no conjunto de treinamento
    model.fit(X_train, y_train)
    model_predictions = model.predict(X_test)
    # Calculo as metricas que eu quero
    # São elas:
    # Mean Error
    # Mean Absolute Error
    # Standard Deviation
    # Correlation
    
    SBP_error = y_test[:, 0] - model_predictions[:, 0]
    SBP_ME.append(np.mean(SBP_error))
    SBP_MAE.append(mean_absolute_error(y_test[:, 0], model_predictions[:, 0]))
    SBP_STD.append(np.std(model_predictions[:, 0]))
    SBP_CEP5.append(cumulative_error_percentage(model_predictions[:, 0], 5))
    SBP_CEP10.append(cumulative_error_percentage(model_predictions[:, 0], 10))
    SBP_CEP15.append(cumulative_error_percentage(model_predictions[:, 0], 15))
    # SBP_CORR.append()
    # SBP_RMSE.append(mean_squared_error(y_test[:, 0], model_predictions[:, 0]))

    DBP_error = y_test[:, 1] - model_predictions[:, 1]
    DBP_ME.append(np.mean(y_test[:, 1] - model_predictions[:, 1]))
    DBP_MAE.append(mean_absolute_error(y_test[:, 1], model_predictions[:, 1]))
    DBP_STD.append(np.std(model_predictions[:, 1]))
    DBP_CEP5.append(cumulative_error_percentage(model_predictions[:, 1], 5))
    DBP_CEP10.append(cumulative_error_percentage(model_predictions[:, 1], 10))
    DBP_CEP15.append(cumulative_error_percentage(model_predictions[:, 1], 15))
    # DBP_CORR.append()
    # DBP_RMSE.append(mean_squared_error(y_test[:, 1], model_predictions[:, 1]))
    
    MBP_error = y_test[:, 2] - model_predictions[:, 2]
    MBP_ME.append(np.mean(y_test[:, 2] - model_predictions[:, 2]))
    MBP_MAE.append(mean_absolute_error(y_test[:, 2], model_predictions[:, 2]))
    MBP_STD.append(np.std(model_predictions[:, 2]))
    MBP_CEP5.append(cumulative_error_percentage(model_predictions[:, 2], 5))
    MBP_CEP10.append(cumulative_error_percentage(model_predictions[:, 2], 10))
    MBP_CEP15.append(cumulative_error_percentage(model_predictions[:, 2], 15))
    # DBP_CORR.append()
    # MBP_RMSE.append(mean_squared_error(y_test[:, 2], model_predictions[:, 2]))
    
# Atribui as métricas aos indices do dataframe    
metrics['SBP(ME)']['LiR'] = np.mean(SBP_ME)
metrics['SBP(MAE)']['LiR'] = np.mean(SBP_MAE)
metrics['SBP(STD)']['LiR'] = np.mean(SBP_STD)
# metrics['SBP(RMSE)']['LiR'] = np.mean(SBP_RMSE)

metrics['DBP(ME)']['LiR'] = np.mean(DBP_ME)
metrics['DBP(MAE)']['LiR'] = np.mean(DBP_MAE)
metrics['DBP(STD)']['LiR'] = np.mean(DBP_STD)
# metrics['DBP(RMSE)']['LiR'] = np.mean(DBP_RMSE)

metrics['MBP(ME)']['LiR'] = np.mean(MBP_ME)
metrics['MBP(MAE)']['LiR'] = np.mean(MBP_MAE)
metrics['MBP(STD)']['LiR'] = np.mean(MBP_STD)
# metrics['MBP(RMSE)']['LiR'] = np.mean(MBP_RMSE)

metrics

,SBP(ME),SBP(MAE),SBP(STD),DBP(ME),DBP(MAE),DBP(STD),MBP(ME),MBP(MAE),MBP(STD)
LiR,-0.0779299,16.449,5.10085,-0.058844,11.1478,2.34393,-0.075858,11.4875,2.64325
DT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
RF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
cumulative_error_percentage(SBP_error, 5)

64.57345971563981

### Train and evaluate Decision Tree Regressor

In [51]:
# Anotações
# Y[:, 0] => Systolic Blood Pressure
# Y[:, 1] => Diastolic Blood Pressure

# Declaração dos vetores para armazenar as métricas
SBP_ME = []
SBP_MAE = []
SBP_STD = []
SBP_CORR = []
SBP_RMSE = []

DBP_ME = []
DBP_MAE = []
DBP_STD = []
DBP_CORR = []
# DBP_RMSE = []

MBP_ME = []
MBP_MAE = []
MBP_STD = []
MBP_CORR = []
# MBP_RMSE = []

# Aqui a gente deve especificar o modelo a ser testado
model = DecisionTreeRegressor(random_state=0)

# Definição do conjunto de validação cruzada pelo metodo KFold
cv = KFold(n_splits=5) #  random_state=42, shuffle=False

for train_index, test_index in cv.split(X):
    # Separo os conjuntos de treino e de teste
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], Y[train_index], Y[test_index]
    # Treino o modelo no conjunto de treinamento
    model.fit(X_train, y_train)
    model_predictions = model.predict(X_test)
    # Calculo as metricas que eu quero
    # São elas:
    # Mean Error
    # Mean Absolute Error
    # Standard Deviation
    
    SBP_error = y_test[:, 0] - model_predictions[:, 0]
    SBP_ME.append(np.mean(SBP_error))
    SBP_MAE.append(mean_absolute_error(y_test[:, 0], model_predictions[:, 0]))
    SBP_STD.append(np.std(model_predictions[:, 0]))
    # SBP_STD.append(np.sqrt(np.mean(abs(SBP_error))**2))
    # SBP_CORR.append()
    # SBP_RMSE.append(mean_squared_error(y_test[:, 0], model_predictions[:, 0]))
    
    DBP_error = y_test[:, 1] - model_predictions[:, 1]
    DBP_ME.append(np.mean(y_test[:, 1] - model_predictions[:, 1]))
    DBP_MAE.append(mean_absolute_error(y_test[:, 1], model_predictions[:, 1]))
    DBP_STD.append(np.std(model_predictions[:, 1]))
    # DBP_CORR.append()
    # DBP_RMSE.append(mean_squared_error(y_test[:, 1], model_predictions[:, 1]))

    MBP_error = y_test[:, 2] - model_predictions[:, 2]
    MBP_ME.append(np.mean(y_test[:, 2] - model_predictions[:, 2]))
    MBP_MAE.append(mean_absolute_error(y_test[:, 2], model_predictions[:, 2]))
    MBP_STD.append(np.std(model_predictions[:, 2]))
    # DBP_CORR.append()
    # MBP_RMSE.append(mean_squared_error(y_test[:, 2], model_predictions[:, 2]))
    
# Atribui as métricas aos indices do dataframe    
metrics['SBP(ME)']['DT'] = np.mean(SBP_ME)
metrics['SBP(MAE)']['DT'] = np.mean(SBP_MAE)
metrics['SBP(STD)']['DT'] = np.mean(SBP_STD)
# metrics['SBP(RMSE)']['DT'] = np.mean(SBP_RMSE)

metrics['DBP(ME)']['DT'] = np.mean(DBP_ME)
metrics['DBP(MAE)']['DT'] = np.mean(DBP_MAE)
metrics['DBP(STD)']['DT'] = np.mean(DBP_STD)
# metrics['DBP(RMSE)']['DT'] = np.mean(DBP_RMSE)

metrics['MBP(ME)']['DT'] = np.mean(MBP_ME)
metrics['MBP(MAE)']['DT'] = np.mean(MBP_MAE)
metrics['MBP(STD)']['DT'] = np.mean(MBP_STD)
# metrics['MBP(RMSE)']['DT'] = np.mean(MBP_RMSE)
metrics

,SBP(ME),SBP(MAE),SBP(STD),DBP(ME),DBP(MAE),DBP(STD),MBP(ME),MBP(MAE),MBP(STD)
LiR,-0.0779299,16.449,5.10085,-0.058844,11.1478,2.34393,-0.075858,11.4875,2.64325
DT,-0.890263,22.7203,22.1899,-0.686654,15.4866,15.3265,-0.763902,15.8219,15.3922
RF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Train and evaluate Random Forest Regressor

In [53]:
# Anotações
# Y[:, 0] => Systolic Blood Pressure
# Y[:, 1] => Diastolic Blood Pressure

import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

# Declaração dos vetores para armazenar as métricas
SBP_ME = []
SBP_MAE = []
SBP_STD = []
SBP_CORR = []
# SBP_RMSE = []

DBP_ME = []
DBP_MAE = []
DBP_STD = []
DBP_CORR = []
# DBP_RMSE = []

MBP_ME = []
MBP_MAE = []
MBP_STD = []
MBP_CORR = []
# MBP_RMSE = []

# Aqui a gente deve especificar o modelo a ser testado
model = RandomForestRegressor(n_estimators = 100, random_state = 42)

# Definição do conjunto de validação cruzada pelo metodo KFold
cv = KFold(n_splits=5) #  random_state=42, shuffle=False

for train_index, test_index in cv.split(X):
    # Separo os conjuntos de treino e de teste
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], Y[train_index], Y[test_index]
    # Treino o modelo no conjunto de treinamento
    model.fit(X_train, y_train)
    model_predictions = model.predict(X_test)
    # Calculo as metricas que eu quero
    # São elas:
    # Mean Error
    # Mean Absolute Error
    # Standard Deviation
    
    SBP_error = y_test[:, 0] - model_predictions[:, 0]
    SBP_ME.append(np.mean(SBP_error))
    SBP_MAE.append(mean_absolute_error(y_test[:, 0], model_predictions[:, 0]))
    SBP_STD.append(np.std(model_predictions[:, 0]))
    # SBP_STD.append(np.sqrt(np.mean(abs(SBP_error))**2))
    # SBP_CORR.append()
    # SBP_RMSE.append(mean_squared_error(y_test[:, 0], model_predictions[:, 0]))

    DBP_error = y_test[:, 1] - model_predictions[:, 1]
    DBP_ME.append(np.mean(y_test[:, 1] - model_predictions[:, 1]))
    DBP_MAE.append(mean_absolute_error(y_test[:, 1], model_predictions[:, 1]))
    DBP_STD.append(np.std(model_predictions[:, 1]))
    # DBP_CORR.append()
    # DBP_RMSE.append(mean_squared_error(y_test[:, 1], model_predictions[:, 1]))

    MBP_error = y_test[:, 2] - model_predictions[:, 2]
    MBP_ME.append(np.mean(y_test[:, 2] - model_predictions[:, 2]))
    MBP_MAE.append(mean_absolute_error(y_test[:, 2], model_predictions[:, 2]))
    MBP_STD.append(np.std(model_predictions[:, 2]))
    # DBP_CORR.append()
    # MBP_RMSE.append(mean_squared_error(y_test[:, 2], model_predictions[:, 2]))
    
# Atribui as métricas aos indices do dataframe    
metrics['SBP(ME)']['RF'] = np.mean(SBP_ME)
metrics['SBP(MAE)']['RF'] = np.mean(SBP_MAE)
metrics['SBP(STD)']['RF'] = np.mean(SBP_STD)
# metrics['SBP(RMSE)']['RF'] = np.mean(SBP_RMSE)

metrics['DBP(ME)']['RF'] = np.mean(DBP_ME)
metrics['DBP(MAE)']['RF'] = np.mean(DBP_MAE)
metrics['DBP(STD)']['RF'] = np.mean(DBP_STD)
# metrics['DBP(RMSE)']['RF'] = np.mean(DBP_RMSE)

metrics['MBP(ME)']['RF'] = np.mean(MBP_ME)
metrics['MBP(MAE)']['RF'] = np.mean(MBP_MAE)
metrics['MBP(STD)']['RF'] = np.mean(MBP_STD)
# metrics['MBP(RMSE)']['RF'] = np.mean(MBP_RMSE)

metrics

,SBP(ME),SBP(MAE),SBP(STD),DBP(ME),DBP(MAE),DBP(STD),MBP(ME),MBP(MAE),MBP(STD)
LiR,-0.0779299,16.449,5.10085,-0.058844,11.1478,2.34393,-0.075858,11.4875,2.64325
DT,-0.890263,22.7203,22.1899,-0.686654,15.4866,15.3265,-0.763902,15.8219,15.3922
RF,-0.752236,16.5116,8.63071,-0.661478,11.4022,4.61682,-0.722468,11.6165,5.34637
AB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
paper_metrics

,SBP(ME),SBP(MAE),SBP(STD),DBP(ME),DBP(MAE),DBP(STD),MBP(ME),MBP(MAE),MBP(STD)
LiR,0.17,16.12,10.03,-0.11,7.04,5.81,0.13,8.89,6.25
DT,-0.71,13.87,15.29,0.17,6.82,8.91,0.02,8.18,9.02
RF,-0.17,10.29,10.35,-0.14,5.77,5.43,0.07,6.38,6.62
AB,0.09,8.22,10.38,0.23,4.17,4.22,-0.02,4.58,5.53


### Train and evaluate AdaBoost Regressor

In [57]:
# Anotações
# Y[:, 0] => Systolic Blood Pressure
# Y[:, 1] => Diastolic Blood Pressure

import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

# Declaração dos vetores para armazenar as métricas
SBP_ME = []
SBP_MAE = []
SBP_STD = []
SBP_CORR = []
# SBP_RMSE = []

DBP_ME = []
DBP_MAE = []
DBP_STD = []
DBP_CORR = []
# DBP_RMSE = []

# Aqui a gente deve especificar o modelo a ser testado
sbp_model = AdaBoostRegressor(random_state=0, n_estimators=200)
dbp_model = AdaBoostRegressor(random_state=0, n_estimators=200)
abp_model = AdaBoostRegressor(random_state=0, n_estimators=200)

# Definição do conjunto de validação cruzada pelo metodo KFold
cv = KFold(n_splits=5) #  random_state=42, shuffle=False

for train_index, test_index in cv.split(X):
    # Separo os conjuntos de treino e de teste
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], Y[train_index], Y[test_index]
    # Treino o modelo no conjunto de treinamento
    sbp_model.fit(X_train, y_train[:, 0])
    dbp_model.fit(X_train, y_train[:, 1])
    abp_model.fit(X_train, y_train[:, 2])
    
    sbp_model_predictions = sbp_model.predict(X_test)
    dbp_model_predictions = dbp_model.predict(X_test)
    abp_model_predictions = abp_model.predict(X_test)
    # Calculo as metricas que eu quero
    # São elas:
    # Mean Error
    # Mean Absolute Error
    # Standard Deviation
    
    SBP_error = y_test[:, 0] - sbp_model_predictions
    SBP_ME.append(np.mean(SBP_error))
    SBP_MAE.append(mean_absolute_error(y_test[:, 0], sbp_model_predictions))
    SBP_STD.append(np.std(sbp_model_predictions))
    # SBP_CORR.append()
    # SBP_RMSE.append(mean_squared_error(y_test[:, 0], sbp_model_predictions))

    DBP_error = y_test[:, 1] - dbp_model_predictions
    DBP_ME.append(np.mean(y_test[:, 1] - dbp_model_predictions))
    DBP_MAE.append(mean_absolute_error(y_test[:, 1], dbp_model_predictions))
    DBP_STD.append(np.std(dbp_model_predictions))
    # DBP_CORR.append()
    # DBP_RMSE.append(mean_squared_error(y_test[:, 1], model_predictions[:, 1]))
    
    MBP_error = y_test[:, 2] - abp_model_predictions
    MBP_ME.append(np.mean(y_test[:, 2] - abp_model_predictions))
    MBP_MAE.append(mean_absolute_error(y_test[:, 2], abp_model_predictions))
    MBP_STD.append(np.std(abp_model_predictions))
    # DBP_CORR.append()
    # MBP_RMSE.append(mean_squared_error(y_test[:, 2], model_predictions[:, 2]))
    
# Atribui as métricas aos indices do dataframe    
metrics['SBP(ME)']['AB'] = np.mean(SBP_ME)
metrics['SBP(MAE)']['AB'] = np.mean(SBP_MAE)
metrics['SBP(STD)']['AB'] = np.mean(SBP_STD)
# metrics['SBP(RMSE)']['RF'] = np.mean(SBP_RMSE)

metrics['DBP(ME)']['AB'] = np.mean(DBP_ME)
metrics['DBP(MAE)']['AB'] = np.mean(DBP_MAE)
metrics['DBP(STD)']['AB'] = np.mean(DBP_STD)
# metrics['DBP(RMSE)']['RF'] = np.mean(DBP_RMSE)

metrics['MBP(ME)']['AB'] = np.mean(MBP_ME)
metrics['MBP(MAE)']['AB'] = np.mean(MBP_MAE)
metrics['MBP(STD)']['AB'] = np.mean(MBP_STD)
# metrics['MBP(RMSE)']['RF'] = np.mean(MBP_RMSE)

metrics

,SBP(ME),SBP(MAE),SBP(STD),DBP(ME),DBP(MAE),DBP(STD),MBP(ME),MBP(MAE),MBP(STD)
LiR,-0.0779299,16.449,5.10085,-0.058844,11.1478,2.34393,-0.075858,11.4875,2.64325
DT,-0.890263,22.7203,22.1899,-0.686654,15.4866,15.3265,-0.763902,15.8219,15.3922
RF,-0.752236,16.5116,8.63071,-0.661478,11.4022,4.61682,-0.722468,11.6165,5.34637
AB,-6.80886,17.7116,5.89868,-13.7179,17.8614,3.98125,-4.78034,13.0434,4.25503


In [58]:
paper_metrics

,SBP(ME),SBP(MAE),SBP(STD),DBP(ME),DBP(MAE),DBP(STD),MBP(ME),MBP(MAE),MBP(STD)
LiR,0.17,16.12,10.03,-0.11,7.04,5.81,0.13,8.89,6.25
DT,-0.71,13.87,15.29,0.17,6.82,8.91,0.02,8.18,9.02
RF,-0.17,10.29,10.35,-0.14,5.77,5.43,0.07,6.38,6.62
AB,0.09,8.22,10.38,0.23,4.17,4.22,-0.02,4.58,5.53
